In [12]:
#####設定讀取資料的年份與時段#####

year = 104    #110年或104年
rush_hour = "晨峰"    #晨峰或昏峰

################################

import pandas as pd

df= pd.read_excel("TRTS-4S屏柵線.xlsx", sheet_name=None)
sheet1 = df["節點編號"]
real_flow = df["實際流量"]
real_flow = real_flow.set_index("group")


ue_results = pd.read_csv(f"{year}{rush_hour}_UE_results.dat", sep="\t")
ue_results.rename(columns={"tailNode": "A", "headNode": "B"}, inplace=True)

ue_results["link_name"] = ue_results.apply(lambda row: f"{int(row['A'])}_{int(row['B'])}", axis=1)

#look up UE_flow in ue_results to add a new column to sheet1 and sheet2
#for sheet2, we need to find the corresponding link name for A1, B1 and A2, B2
#that is, there are two link to be found for each row in sheet2
#then we need to add up the two link's UE_flow to get the total flow for this row
#so that we can compare with AM flow
sheet1["link_name1"] = sheet1.apply(lambda row: f"{int(row['A1'])}_{int(row['B1'])}", axis=1)
sheet1["link_name2"] = sheet1.apply(lambda row: f"{int(row['A2'])}_{int(row['B2'])}", axis=1)
sheet1 = sheet1.merge(ue_results[["link_name", "UE_flow", "capacity"]], left_on="link_name1", right_on="link_name", how="left")
sheet1 = sheet1.merge(ue_results[["link_name", "UE_flow", "capacity"]], left_on="link_name2", right_on="link_name", how="left", suffixes=('_1', '_2'))
#calculate the total flow and capacity for this row

#reserve the link,AM,UE_flow
sheet1 = sheet1[["group","link", "UE_flow_1", "UE_flow_2"]]

group_names = sheet1['group'].unique()
groups = {i: {'UE_flow_1': 0, 'UE_flow_2': 0} for i in group_names}
for index, row in sheet1.iterrows():
    group = row['group']
    groups[group]['UE_flow_1'] += row['UE_flow_1']
    groups[group]['UE_flow_2'] += row['UE_flow_2']

# UE_flow_1 代表(往南/東)流量， UE_flow_2 代表(往北/西)流量

for group, flows in groups.items():
    print(f"""【{group}】 
    UE flow 往南/東 = {round(flows['UE_flow_1'])}, diff = {(flows['UE_flow_1'] - real_flow.loc[group, '南/東']) / real_flow.loc[group, '南/東']* 100:.2f}%
    UE flow 往北/西 = {round(flows['UE_flow_2'])}, diff = {(flows['UE_flow_2'] - real_flow.loc[group, '北/西']) / real_flow.loc[group, '北/西']* 100:.2f}%
   """)

【SL8】 
    UE flow 往南/東 = 26027, diff = 0.09%
    UE flow 往北/西 = 32703, diff = -20.98%
   
【SL1】 
    UE flow 往南/東 = 49780, diff = 16.63%
    UE flow 往北/西 = 51207, diff = 20.88%
   
【SL6】 
    UE flow 往南/東 = 5182, diff = -7.23%
    UE flow 往北/西 = 3733, diff = -1.18%
   
【SL7】 
    UE flow 往南/東 = 41416, diff = 4.41%
    UE flow 往北/西 = 27856, diff = 17.87%
   
【SL5】 
    UE flow 往南/東 = 8273, diff = -8.54%
    UE flow 往北/西 = 16986, diff = -5.69%
   


In [ ]:
# sheet1 sorted by group and link
sheet1.sort_values(by=['group', 'link'], inplace=True)
pd.set_option('display.max_rows', None)
sheet1[["group", "link", "UE_flow_1", "UE_flow_2"]]
#show all the results in a table